<a href="https://colab.research.google.com/github/isc-patrick/Edifai-AI-Class/blob/main/ISC_Patient_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creation of a patient clinical visit dataset for the GenAI class

1. We will use Synthea to create our initial dataset as it provides a realistic baseline of data that we can massage into what we want.



# Tables

__Patients__
Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,MIDDLE,LAST,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,FIPS,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,INCOME   
1a1e8d0d-7c11-b28d-a5ad-8dd4eec51217,2006-05-26,,999-23-6234,S99940167,,Mr.,Dante562,Quincy153,Hudson301,,,,white,nonhispanic,M,Weymouth  Massachusetts  US,164 Lehner Alley,Newton,Massachusetts,Middlesex County,25017,02465,42.34704045970836,-71.17547973514273,63155.27,23049.50,991816

__Encounter__
Id,START,STOP,PATIENT,ORGANIZATION,PROVIDER,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION   
298300f0-6d5b-321a-fc45-4ff18447c17a,2014-05-23T22:22:59Z,2014-05-23T22:37:59Z,1a1e8d0d-7c11-b28d-a5ad-8dd4eec51217,65eda534-1bdc-3192-8d02-1c3764447790,b7934fdb-7b65-33b1-b6ea-f270a78fbd93,26aab0cd-6aba-3e1b-ac5b-05c8867e762c,wellness,410620009,Well child visit (procedure),136.80,704.20,0.00,,

__Observation__
DATE,PATIENT,ENCOUNTER,CATEGORY,CODE,DESCRIPTION,VALUE,UNITS,TYPE
2015-05-29T22:22:59Z,1a1e8d0d-7c11-b28d-a5ad-8dd4eec51217,08e62fdb-a8ef-3699-de96-d98fbf90fa13,vital-signs,8302-2,Body Height,130.3,cm,numeric

__Conditions__
START,STOP,PATIENT,ENCOUNTER,SYSTEM,CODE,DESCRIPTION  
2014-05-23,2015-05-29,1a1e8d0d-7c11-b28d-a5ad-8dd4eec51217,298300f0-6d5b-321a-fc45-4ff18447c17a,SNOMED-CT,314529007,Medication review due (situation)

__CarePlans__
Id,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,REASONCODE,REASONDESCRIPTION  
67e83af7-27a0-7743-c1c5-dfdf5fbc7a8a,2017-07-17,2017-10-25,1a1e8d0d-7c11-b28d-a5ad-8dd4eec51217,06888dcc-a38c-d2eb-a7f6-416d63502edc,773513001,Physiotherapy care plan (record artifact),30832001,Rupture of patellar tendon (disorder)

__Medications__
START,STOP,PATIENT,PAYER,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION  
2015-06-06T00:52:58Z,2015-06-06T00:52:58Z,1a1e8d0d-7c11-b28d-a5ad-8dd4eec51217,26aab0cd-6aba-3e1b-ac5b-05c8867e762c,d82c6feb-1405-38ba-e526-59b3842e07fd,1535362,sodium fluoride 0.0272 MG/MG Oral Gel,129.94,0.00,1,129.94,66383009,Gingivitis (disorder)

__Procedures__
START,STOP,PATIENT,ENCOUNTER,SYSTEM,CODE,DESCRIPTION,BASE_COST,REASONCODE,REASONDESCRIPTION  
2015-05-29T22:22:59Z,2015-05-29T22:37:59Z,1a1e8d0d-7c11-b28d-a5ad-8dd4eec51217,08e62fdb-a8ef-3699-de96-d98fbf90fa13,SNOMED-CT,430193006,Medication reconciliation (procedure),248.91,,

Patients 1:N Encounters  
Encounters 1:N Observations  
Encounters 1:N Conditions   
Encounters 1:N CarePlans  
Encounters 1:N Medications.  
Encounters 1:N Procedures

..





In [ ]:
!mkdir -p mydata/synthea

import os
os.makedirs('mydata/', exist_ok=True)

In [ ]:
from huggingface_hub import login
from google.colab import userdata
import os

login(userdata.get('HUGGING_FACE_TOKEN'))

repo_id = "patjs/patient1"
ROOT_DIR = "mydata"

from huggingface_hub import HfApi
api = HfApi()

def load_file(filename, foldername):
    filepath =  api.hf_hub_download(repo_id=repo_id, filename=filename, subfolder=foldername, repo_type="dataset")

    with open(filepath, 'r') as fi:
      data = fi.read()

    local_dir = os.path.join(ROOT_DIR,foldername)
    # create the dir if it doesn't exist
    if not os.path.exists(local_dir):
      os.makedirs(local_dir, exist_ok=True)

    local_path = os.path.join(ROOT_DIR,foldername,filename)

    with open(local_path, 'w') as fo:
      print(len(data))
      fo.write(data)

    return local_path

def get_file_path(filename, foldername):
    filepath =  api.hf_hub_download(repo_id=repo_id, filename=filename, subfolder=foldername, repo_type="dataset")
    return filepath

# def save_folder(file_obj, file_name):

#     api.upload_folder(
#       folder_path="/content/sample_data/synthea",
#       path_in_repo="/synthea",
#       repo_id="patjs/patient1",
#       repo_type="dataset",
#     )

def save_file(file_obj, file_name):
  print(file_name, repo_id)
  api.upload_file(
      path_or_fileobj=file_obj,
      path_in_repo=file_name,
      repo_id=repo_id,
      repo_type="dataset",
  )

#print(load_file('patients.csv', 'synthea'))

In [ ]:
import pandas as pd
import os

def convert_time_to_datetime(df: pd.DataFrame, time_col: str) -> pd.DataFrame:
    # ensure column is datettime type
    df[time_col] = pd.to_datetime(df[time_col])
    df[time_col] = df[time_col].dt.date
    return df

def load_csvs_to_pandas(dirpath: str, filenames: list[str],
                        from_hugging = True) -> dict[str, pd.DataFrame]:
    df_dict = {}
    for filename in filenames:
        if from_hugging:
            filepath = load_file(filename, dirpath)
        else:
            filepath = os.path.join(dirpath, filename)
        print(f"Filepath={filepath}")
        df = pd.read_csv(filepath)
        #remove the .csv from filename for key
        key = filename[:-4]
        df_dict[key] = df
    return df_dict

def create_id_map(df: pd.DataFrame, id_col: str) -> dict[str, int]:
    """ Create a map from id to the index"""
    # Create a column that contains the index data and convert to dict
    df = df.reset_index()
    map = dict(zip(df[id_col], df['index']))
    return map

def replace_complex_ids(df: pd.DataFrame, id_map: dict[str, int], col_name: str) -> pd.DataFrame:
    """
    Replace complex ids with integer indices
    """
    df[col_name] = df[col_name].map(id_map)
    return df

def flatten_rows_to_string(df, group_column, value_column, separator=','):
    """
    Flattens rows within groups into a single string in a new column.

    Args:
      df: The Pandas DataFrame.
      group_column: The column to group by.
      value_column: The column containing values to concatenate.
      separator: The string separator for concatenating values (default: ',').

    Returns:
      A new DataFrame with an additional column containing the flattened strings.
    """
    # Group by the specified column and aggregate with join
    grouped = df.groupby(group_column)[value_column].agg(lambda x: separator.join(map(str, x))).reset_index()

    # Rename the aggregated column
    #grouped.rename(columns={value_column: f'{value_column}_flat'}, inplace=True)

    return grouped

def count_non_null_pairs(df: pd.DataFrame, col1: str, col2: str) -> int:
    # Find the number of times where col1 is not null and col2 is null as a percent of total rows
    return len(df[(df[col1].notnull()) & (df[col2].isnull())]) / len(df)

def get_unique_vals(df: pd.DataFrame, col: str, col2: str) -> set[str]:
    return set(df[col,col2].unique())

def process_table(df_dict: dict[str, pd.DataFrame], merged_df: pd.DataFrame, table_name: str,
                  prefix: str, group_column: str, value_columns: list[str]) -> pd.DataFrame:
    """
    Process a specified table by filtering, concatenating specified columns,
    flattening rows, and merging with the main DataFrame.

    Args:
        df_dict: Dictionary of DataFrames containing the tables.
        merged_df: The main DataFrame to merge the table into.
        table_name: The name of the table to process.
        prefix: The prefix to add to the concatenated description.
        group_column: The column to group by.
        value_columns: The columns containing values to concatenate.

    Returns:
        The merged DataFrame with the specified table processed and added.
    """
    # Filter out all Surveys if table is observations
    if table_name == 'observations':
        df = df_dict[table_name][df_dict[table_name]['CATEGORY'] != 'survey']
    else:
        df = df_dict[table_name]

    # Concatenate specified columns
    df['DESCRIPTION'] = f"{prefix}: " + df[value_columns].astype(str).agg(' '.join, axis=1)

    # Flatten rows within groups into a single string in a new column
    df = flatten_rows_to_string(df, group_column, 'DESCRIPTION', ', ')

    # Drop columns that are no longer needed
    df = df[[group_column, 'DESCRIPTION']]

    # Merge with the main DataFrame and drop the group column used for joining
    merged_df = pd.merge(merged_df, df, left_on='ENCOUNTER_ID', right_on=group_column, how='left', suffixes=('', f'_{table_name.upper()}'))
    merged_df = merged_df.drop(columns=[group_column])
    return merged_df

def create_normalized_encounter(df_dict: dict[str, pd.DataFrame]) -> pd.DataFrame:
    """
    Create a normalized encounter DataFrame from the given CSVs
    Should include the patient id, all fields from encounter, and the Description field
    from each table concatenated in a single column per encounter

    """
    # First create a normalized patient and encounter DataFrame, only including
    # the patient id and all fields from encounter
    patient_df = df_dict['patients'][['Id']].set_index('Id')
    encounter_df = df_dict['encounters']
    encounter_df = encounter_df.drop(columns=['ORGANIZATION', 'PROVIDER', 'PAYER'])

    # I used SQL, which most students should know, to generate the pandas merge
    # SELECT patient.id, encounter.* FROM patients, encounters
    # WHERE patients.id = encounters.patient_id
    merged_df = pd.merge(patient_df, encounter_df, left_index=True, right_on='PATIENT')


    # Change name of ID to Encounter ID and Patient to Patient ID
    merged_df = merged_df.rename(columns={'Id': 'ENCOUNTER_ID', 'PATIENT': 'PATIENT_ID'})

    print(f"total patients: {len(patient_df)}")
    print(f"total patient encounters: {len(merged_df)}")

    # Merge OBSERVATIONS
    merged_df = process_table(df_dict, merged_df, 'observations', 'OBSERVATIONS', 'ENCOUNTER', ['DESCRIPTION', 'VALUE', 'UNITS'])

    # Merge CONDITIONS
    merged_df = process_table(df_dict, merged_df, 'conditions', 'CONDITIONS', 'ENCOUNTER', ['DESCRIPTION'])

    # Mergemedications
    merged_df = process_table(df_dict, merged_df, 'medications', 'MEDICATIONS', 'ENCOUNTER', ['DESCRIPTION'])

    # Merge procedures
    merged_df = process_table(df_dict, merged_df, 'procedures', 'PROCEDURES', 'ENCOUNTER', ['DESCRIPTION'])

    return merged_df


In [ ]:
path_small = 'synthea'

filenames = ['patients.csv', 'encounters.csv', 'observations.csv', 'conditions.csv',
             'medications.csv', 'procedures.csv']

df_dict = load_csvs_to_pandas(path_small, filenames)
normalized_encounter_df = create_normalized_encounter(df_dict)

# Convert time columns to datetime
normalized_encounter_df = convert_time_to_datetime(normalized_encounter_df, 'START')
normalized_encounter_df = convert_time_to_datetime(normalized_encounter_df, 'STOP')

# Map GUIDs to ints
patient_id_map = create_id_map(df_dict['patients'], 'Id')
encounter_id_map = create_id_map(df_dict['encounters'], 'Id')

final_df = replace_complex_ids(normalized_encounter_df, patient_id_map, 'PATIENT_ID')
final_df = replace_complex_ids(final_df, encounter_id_map, 'ENCOUNTER_ID')

# Save final df
final_df.to_csv('final.csv', index=False)
final_df.to_json('final.json', orient='records', lines=True,date_format='iso')

# Save to hugging face
with open('final.csv', 'rb') as f:
  save_file(f, 'final.csv')


patients.csv:   0%|          | 0.00/344k [00:00<?, ?B/s]

343530
Filepath=mydata/synthea/patients.csv


encounters.csv:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

27278705
Filepath=mydata/synthea/encounters.csv


observations.csv:   0%|          | 0.00/190M [00:00<?, ?B/s]

...

19743521
Filepath=mydata/synthea/medications.csv


procedures.csv:   0%|          | 0.00/44.5M [00:00<?, ?B/s]

44537314
Filepath=mydata/synthea/procedures.csv
total patients: 1147
total patient encounters: 81482


<ipython-input-12-3d4a15c11641>:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DESCRIPTION'] = f"{prefix}: " + df[value_columns].astype(str).agg(' '.join, axis=1)


final.csv patjs/patient1
